In [ ]:
import os, glob, numpy as np, cv2, random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from scipy.stats import pearsonr
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical

# ------------------ Paths ------------------
IMG_DIR = r"/content/drive/MyDrive/Dataset/images"
ANN_DIR = r"/content/drive/MyDrive/Dataset/annotations"


# ------------------ Load dataset ------------------
def load_dataset(limit=None):
    X, y_cls, y_val, y_aro = [], [], [], []
    img_files = glob.glob(os.path.join(IMG_DIR, "*.jpg"))
    if limit: img_files = img_files[:limit]
    for img_path in img_files:
        stem = os.path.splitext(os.path.basename(img_path))[0]
        try:
            exp = np.load(os.path.join(ANN_DIR, f"{stem}_exp.npy"))
            val = np.load(os.path.join(ANN_DIR, f"{stem}_val.npy"))
            aro = np.load(os.path.join(ANN_DIR, f"{stem}_aro.npy"))
        except FileNotFoundError:
            continue
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (96, 96))
        X.append(img/255.0)
        y_cls.append(int(exp))
        y_val.append(float(val))
        y_aro.append(float(aro))
    X = np.array(X, dtype="float32")
    y_cls = to_categorical(y_cls, num_classes=8)
    y_val, y_aro = np.array(y_val), np.array(y_aro)
    return X, y_cls, y_val, y_aro

print("Loading data…")
X, y_cls, y_val, y_aro = load_dataset()   # remove limit for full run
print("Total samples:", len(X))

# Train/val split
X_train, X_test, y_cls_train, y_cls_test, y_val_train, y_val_test, y_aro_train, y_aro_test = \
    train_test_split(X, y_cls, y_val, y_aro, test_size=0.2, random_state=42)

# ------------------ Models ------------------
def build_custom():
    m = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(96,96,3)),
        layers.MaxPooling2D(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(8, activation='softmax')
    ])
    m.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return m

def build_mobilenet():
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96,96,3))
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    out = layers.Dense(8, activation='softmax')(x)
    m = models.Model(base.input, out)
    m.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return m

# ------------------ Train classification ------------------
def quick_train(model, name):
    print(f"\nTraining {name}")
    model.fit(X_train, y_cls_train, epochs=5, batch_size=16,
              validation_split=0.1, verbose=1)
    preds = model.predict(X_test)
    y_pred_cls = preds.argmax(1)
    y_true_cls = y_cls_test.argmax(1)
    acc = accuracy_score(y_true_cls, y_pred_cls)
    f1  = f1_score(y_true_cls, y_pred_cls, average='weighted')
    print(f"{name} Accuracy: {acc:.3f}  F1: {f1:.3f}")
    return acc, f1

acc1,f1_1 = quick_train(build_custom(), "CustomCNN")
acc2,f1_2 = quick_train(build_mobilenet(), "MobileNetV2")

print("\nComparison => CustomCNN Acc:{:.3f} F1:{:.3f} | MobileNetV2 Acc:{:.3f} F1:{:.3f}"
      .format(acc1,f1_1,acc2,f1_2))

# ------------------ Regression heads for valence/arousal ------------------
def build_regressor():
    m = models.Sequential([
        layers.Conv2D(16,(3,3),activation='relu',input_shape=(96,96,3)),
        layers.MaxPooling2D(),
        layers.Conv2D(32,(3,3),activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(32,activation='relu'),
        layers.Dense(2)  # [valence, arousal]
    ])
    m.compile(optimizer='adam', loss='mse')
    return m

y_reg_train = np.stack([y_val_train, y_aro_train], axis=1)
y_reg_test  = np.stack([y_val_test,  y_aro_test ], axis=1)

reg = build_regressor()
print("\nTraining Valence/Arousal regressor")
reg.fit(X_train, y_reg_train, epochs=5, batch_size=16,
        validation_split=0.1, verbose=1)
pred = reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_reg_test, pred))
corr_v = pearsonr(y_val_test, pred[:,0])[0]
corr_a = pearsonr(y_aro_test, pred[:,1])[0]
print(f"Valence/Arousal RMSE:{rmse:.3f}  Corr V:{corr_v:.3f}  Corr A:{corr_a:.3f}")


Loading data…
Total samples: 3999


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training CustomCNN
Epoch 1/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 44s 236ms/step - accuracy: 0.1433 - loss: 2.2172 - val_accuracy: 0.2188 - val_loss: 2.0375
Epoch 2/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 40s 223ms/step - accuracy: 0.2680 - loss: 1.9314 - val_accuracy: 0.2844 - val_loss: 1.8790
Epoch 3/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 41s 227ms/step - accuracy: 0.3902 - loss: 1.6705 - val_accuracy: 0.2562 - val_loss: 1.9470
Epoch 4/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 42s 234ms/step - accuracy: 0.5278 - loss: 1.3382 - val_accuracy: 0.2906 - val_loss: 1.9451
Epoch 5/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 41s 227ms/step - accuracy: 0.6692 - loss: 0.9542 - val_accuracy: 0.2656 - val_loss: 2.3540
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step
CustomCNN Accuracy: 0.271  F1: 0.276
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Training MobileNetV2
Epoch 1/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 31s 137ms/step - accuracy: 0.1587 - loss: 2.2594 - val_accuracy: 0.2000 - val_loss: 2.1075
Epoch 2/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 22s 122ms/st

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 18s 93ms/step - loss: 0.1961 - val_loss: 0.1740
Epoch 2/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.1840 - val_loss: 0.1719
Epoch 3/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 16s 91ms/step - loss: 0.1793 - val_loss: 0.1719
Epoch 4/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - loss: 0.1805 - val_loss: 0.1711
Epoch 5/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 16s 91ms/step - loss: 0.1846 - val_loss: 0.1711
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
Valence/Arousal RMSE:0.426  Corr V:0.093  Corr A:0.085


In [ ]:
import os, glob, numpy as np, cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from scipy.stats import pearsonr
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

IMG_DIR = "/content/drive/MyDrive/Dataset/images"
ANN_DIR = "/content/drive/MyDrive/Dataset/annotations"

# ---------- Load Data ----------
def load_dataset(limit=None, size=124):
    X, y_cls, y_val, y_aro = [], [], [], []
    files = glob.glob(os.path.join(IMG_DIR, "*.jpg"))
    if limit: files = files[:limit]
    for p in files:
        stem = os.path.splitext(os.path.basename(p))[0]
        try:
            exp = np.load(os.path.join(ANN_DIR, f"{stem}_exp.npy"))
            val = np.load(os.path.join(ANN_DIR, f"{stem}_val.npy"))
            aro = np.load(os.path.join(ANN_DIR, f"{stem}_aro.npy"))
        except FileNotFoundError:
            continue
        img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (size, size))
        X.append(img/255.)
        y_cls.append(int(exp))
        y_val.append(float(val))
        y_aro.append(float(aro))
    X = np.array(X, dtype="float32")
    y_cls = to_categorical(y_cls, num_classes=8)
    return X, y_cls, np.array(y_val), np.array(y_aro)

print("Loading full dataset …")
X, y_cls, y_val, y_aro = load_dataset(size=124)
print("Total samples:", len(X))

X_train, X_test, y_cls_train, y_cls_test, y_val_train, y_val_test, y_aro_train, y_aro_test = \
    train_test_split(X, y_cls, y_val, y_aro, test_size=0.2, random_state=42)

# ---------- Augmentation ----------
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# ---------- Fine-tunable MobileNet ----------
def build_mobilenet_finetune():
    base = MobileNetV2(weights='imagenet', include_top=False,
                       input_shape=(124,124,3))
    base.trainable = False  # frozen first
    x = layers.GlobalAveragePooling2D()(base.output)
    out = layers.Dense(8, activation='softmax')(x)
    model = models.Model(base.input, out)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model, base

# ---------- Train with staged unfreeze ----------
# ---------------- Train with staged unfreeze (fixed) ----------------
def train_mobilenet():
    model, base = build_mobilenet_finetune()
    es = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    lr_sched = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

    # ✅ manual validation split
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_cls_train, test_size=0.1, random_state=42)

    print("\nStage 1: frozen base")
    model.fit(datagen.flow(X_tr, y_tr, batch_size=32),
              validation_data=(X_val, y_val),
              epochs=5,
              callbacks=[es, lr_sched],
              verbose=1)

    print("\nStage 2: unfreeze top 20 layers")
    for layer in base.layers[-20:]:
        layer.trainable = True
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(datagen.flow(X_tr, y_tr, batch_size=32),
              validation_data=(X_val, y_val),
              epochs=20,
              callbacks=[es, lr_sched],
              verbose=1)
    return model


model = train_mobilenet()

# ---------- Evaluation ----------
preds = model.predict(X_test)
y_pred_cls = preds.argmax(1)
y_true_cls = y_cls_test.argmax(1)
acc = accuracy_score(y_true_cls, y_pred_cls)
f1  = f1_score(y_true_cls, y_pred_cls, average='weighted')
print(f"MobileNetV2 Fine-tuned  Acc:{acc:.3f}  F1:{f1:.3f}")

# ---------- Valence/Arousal Regressor ----------
def build_regressor():
    m = models.Sequential([
        layers.Conv2D(32,(3,3),activation='relu',input_shape=(124,124,3)),
        layers.MaxPooling2D(),
        layers.Conv2D(64,(3,3),activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(64,activation='relu'),
        layers.Dense(2)
    ])
    m.compile(optimizer='adam', loss='mse')
    return m

y_reg_train = np.stack([y_val_train, y_aro_train], axis=1)
y_reg_test  = np.stack([y_val_test,  y_aro_test ], axis=1)

reg = build_regressor()
print("\nTraining Valence/Arousal regressor")
reg.fit(datagen.flow(X_train, y_reg_train, batch_size=32),
        validation_split=0.1,
        epochs=20,
        callbacks=[EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)],
        verbose=1)
pred = reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_reg_test, pred))
corr_v = pearsonr(y_val_test, pred[:,0])[0]
corr_a = pearsonr(y_aro_test, pred[:,1])[0]
print(f"Val/Aro RMSE:{rmse:.3f}  CorrV:{corr_v:.3f}  CorrA:{corr_a:.3f}")


Loading full dataset …
Total samples: 3999


/tmp/ipython-input-881170184.py:55: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False,



Stage 1: frozen base


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 55s 554ms/step - accuracy: 0.1533 - loss: 2.4261 - val_accuracy: 0.2438 - val_loss: 2.1424 - learning_rate: 0.0010
Epoch 2/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 80s 532ms/step - accuracy: 0.2529 - loss: 2.0233 - val_accuracy: 0.2625 - val_loss: 2.1185 - learning_rate: 0.0010
Epoch 3/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 47s 513ms/step - accuracy: 0.3009 - loss: 1.9102 - val_accuracy: 0.2656 - val_loss: 2.0113 - learning_rate: 0.0010
Epoch 4/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 53s 590ms/step - accuracy: 0.3012 - loss: 1.8450 - val_accuracy: 0.2469 - val_loss: 1.9732 - learning_rate: 0.0010
Epoch 5/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 48s 537ms/step - accuracy: 0.3322 - loss: 1.7503 - val_accuracy: 0.2656 - val_loss: 1.9814 - learning_rate: 0.0010

Stage 2: unfreeze top 20 layers
Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 71s 691ms/step - accuracy: 0.2174 - loss: 2.0835 - val_accuracy: 0.2531 - val_loss: 1.9748 - learning_rate: 1.0000e-05
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 60s 662ms/st

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Argument `validation_split` is only supported for tensors or NumPy arrays.Found incompatible type in the input: [<class 'keras.src.legacy.preprocessing.image.NumpyArrayIterator'>]

In [ ]:
##Final

In [ ]:
import os, glob, numpy as np, cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from scipy.stats import pearsonr
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# dataset paths
IMG_DIR = "/content/drive/MyDrive/Dataset/images"
ANN_DIR = "/content/drive/MyDrive/Dataset/annotations"

In [ ]:
def load_dataset(limit=None, size=124):
    X, y_cls, y_val, y_aro = [], [], [], []
    files = glob.glob(os.path.join(IMG_DIR, "*.jpg"))
    if limit: files = files[:limit]
    for p in files:
        stem = os.path.splitext(os.path.basename(p))[0]
        try:
            exp = np.load(os.path.join(ANN_DIR, f"{stem}_exp.npy"))
            val = np.load(os.path.join(ANN_DIR, f"{stem}_val.npy"))
            aro = np.load(os.path.join(ANN_DIR, f"{stem}_aro.npy"))
        except FileNotFoundError:
            continue
        img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (size, size))
        X.append(img/255.)
        y_cls.append(int(exp))
        y_val.append(float(val))
        y_aro.append(float(aro))
    X = np.array(X, dtype="float32")
    y_cls = to_categorical(y_cls, num_classes=8)
    return X, y_cls, np.array(y_val), np.array(y_aro)

print("Loading dataset …")
X, y_cls, y_val, y_aro = load_dataset(size=124)
print("Total samples:", len(X))


Loading dataset …
Total samples: 3999


In [ ]:
X_train, X_test, y_cls_train, y_cls_test, y_val_train, y_val_test, y_aro_train, y_aro_test = \
    train_test_split(X, y_cls, y_val, y_aro, test_size=0.2, random_state=42)

# Augment only training set
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)


In [ ]:
def build_mobilenet_finetune():
    base = MobileNetV2(weights='imagenet', include_top=False,
                       input_shape=(124,124,3))
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    out = layers.Dense(8, activation='softmax')(x)
    model = models.Model(base.input, out)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model, base

def train_mobilenet():
    model, base = build_mobilenet_finetune()
    es = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    lr_sched = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

    # explicit validation split
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_cls_train, test_size=0.1, random_state=42)

    print("\nStage 1: frozen base")
    model.fit(datagen.flow(X_tr, y_tr, batch_size=32),
              validation_data=(X_val, y_val),
              epochs=5, callbacks=[es, lr_sched], verbose=1)

    print("\nStage 2: unfreeze top 20 layers")
    for layer in base.layers[-20:]:
        layer.trainable = True
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(datagen.flow(X_tr, y_tr, batch_size=32),
              validation_data=(X_val, y_val),
              epochs=20, callbacks=[es, lr_sched], verbose=1)
    return model

model = train_mobilenet()


/tmp/ipython-input-759463891.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False,



Stage 1: frozen base


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 56s 556ms/step - accuracy: 0.1715 - loss: 2.5567 - val_accuracy: 0.2344 - val_loss: 2.0666 - learning_rate: 0.0010
Epoch 2/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 48s 535ms/step - accuracy: 0.2438 - loss: 2.0376 - val_accuracy: 0.2719 - val_loss: 1.9945 - learning_rate: 0.0010
Epoch 3/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 46s 512ms/step - accuracy: 0.2859 - loss: 1.8945 - val_accuracy: 0.2750 - val_loss: 1.9758 - learning_rate: 0.0010
Epoch 4/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 47s 523ms/step - accuracy: 0.3080 - loss: 1.8360 - val_accuracy: 0.2844 - val_loss: 1.9667 - learning_rate: 0.0010
Epoch 5/5
90/90 ━━━━━━━━━━━━━━━━━━━━ 45s 504ms/step - accuracy: 0.3234 - loss: 1.8054 - val_accuracy: 0.2906 - val_loss: 1.9812 - learning_rate: 0.0010

Stage 2: unfreeze top 20 layers
Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 69s 657ms/step - accuracy: 0.2224 - loss: 2.0676 - val_accuracy: 0.2500 - val_loss: 1.9674 - learning_rate: 1.0000e-05
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 58s 639ms/st

In [ ]:
preds = model.predict(X_test)
y_pred_cls = preds.argmax(1)
y_true_cls = y_cls_test.argmax(1)
acc = accuracy_score(y_true_cls, y_pred_cls)
f1  = f1_score(y_true_cls, y_pred_cls, average='weighted')
print(f"MobileNetV2 Fine-tuned  Acc:{acc:.3f}  F1:{f1:.3f}")


25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 343ms/step
MobileNetV2 Fine-tuned  Acc:0.296  F1:0.303


In [ ]:
def build_regressor():
    m = models.Sequential([
        layers.Input(shape=(124,124,3)),
        layers.Conv2D(32,(3,3),activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64,(3,3),activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(64,activation='relu'),
        layers.Dense(2)
    ])
    m.compile(optimizer='adam', loss='mse')
    return m

# prepare regression targets
y_reg_train = np.stack([y_val_train, y_aro_train], axis=1)
y_reg_test  = np.stack([y_val_test,  y_aro_test ], axis=1)

reg = build_regressor()
print("\nTraining Valence/Arousal regressor")

# explicit validation set, not validation_split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_reg_train, test_size=0.1, random_state=42)

reg.fit(datagen.flow(X_tr, y_tr, batch_size=32),
        validation_data=(X_val, y_val),
        epochs=20,
        callbacks=[EarlyStopping(monitor='val_loss', patience=4,
                                 restore_best_weights=True)],
        verbose=1)

pred = reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_reg_test, pred))
corr_v = pearsonr(y_val_test, pred[:,0])[0]
corr_a = pearsonr(y_aro_test, pred[:,1])[0]
print(f"Val/Aro RMSE:{rmse:.3f}  CorrV:{corr_v:.3f}  CorrA:{corr_a:.3f}")



Training Valence/Arousal regressor


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 70s 759ms/step - loss: 0.1959 - val_loss: 0.1904
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 66s 732ms/step - loss: 0.1847 - val_loss: 0.1849
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 66s 722ms/step - loss: 0.1813 - val_loss: 0.1853
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 66s 728ms/step - loss: 0.1817 - val_loss: 0.1854
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 82s 724ms/step - loss: 0.1807 - val_loss: 0.1854
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 82s 724ms/step - loss: 0.1774 - val_loss: 0.1851
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 149ms/step
Val/Aro RMSE:0.425  CorrV:0.061  CorrA:0.042
